# Example: Computing Single Source Shortest Paths using Dijkstra’s Algorithm
This example will familiarize students with working with [Dijkstra’s Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)
* Dijkstra’s Algorithm is a single-source shortest path algorithm for a graph with non-negative edge weights. It is greedy and maintains a set of vertices whose shortest distance from the source is known. The algorithm repeatedly selects the vertex `u` from the set of vertices not yet processed 
with the minimum distance `dist[u],` explores the edges incident to `u,` and adds `u` to the set of processed vertices. The algorithm terminates when the set of processed vertices includes all the vertices in the graph. 
* The algorithm is named after Edsger W. Dijkstra, who described it in 1959 (supposedly, it took him 20 minutes while shopping with his lady friend). 

## Setup
This example may use external third-party packages. In the `Include.jl` file, we load our codes to access them in the notebook, set some required paths for this example, and load any required external packages.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-7/L7a/Manifest.toml`


## Task 1: Build an instance of `MySimpleDirectedGraphModel` for an example graph using an edge list
A graph $\mathcal{G}=\left(\mathcal{V},\mathcal{E}\right)$ can be constructed from an [Adjacency Matrix](https://en.wikipedia.org/wiki/Adjacency_matrix) $\mathbf{A}$, which is a $\dim\mathcal{V}\times\dim\mathcal{V}$ square matrix. However, this is only suitable for small graphs because $\mathbf{A}$ has a high memory overhead (if stored as `64-bit` values). 
* For example, consider a graph with $\dim\mathcal{V}$ = 100000 would require `80 GB` of memory to store in the worst case, which is more than most common machines

In [2]:
path_to_edge_file = joinpath(_PATH_TO_DATA, "SimpleGraph.txt");

Next, let's load build a dictionary of `edges,` where the data for the edges (source id, target id, and weight) is stored in a `MyGraphEdgeModel` model. 
* We utilize the `readedgesfile` function from `src/Files.jl` to read the edge data. This function requires the path to the edges file and information about the delimiter and comment characters. It returns a dictionary that holds instances of `MyGraphEdgeModel`

In [28]:
myedges = readedgesfile(path_to_edge_file, delim=',', comment='#');
myedges

Dict{Int64, MyGraphEdgeModel} with 7 entries:
  0 => MyGraphEdgeModel(0, 1, 2, 10.0)
  4 => MyGraphEdgeModel(4, 3, 5, 6.0)
  5 => MyGraphEdgeModel(5, 4, 6, 1.0)
  6 => MyGraphEdgeModel(6, 5, 4, 1.0)
  2 => MyGraphEdgeModel(2, 2, 3, 2.0)
  3 => MyGraphEdgeModel(3, 2, 4, 100.0)
  1 => MyGraphEdgeModel(1, 1, 3, 100.0)

Finally, now that we have the `myedges` dictionary, we can build a graph instance. Since this is a directed graph, we'll construct a `MySimpleDirectedGraphModel` instance using it's `build(...)` method in the `src/Factory.jl` file.

In [4]:
dag = build(MySimpleDirectedGraphModel, myedges);

## Task 2: Use our  implementation of Dijkstra’s Algorithm to visit the nodes of the `dag`

We've implemented the `computeshortestpaths(...)` function in `src/Search.jl` file that can be used to compute the shortest path. Notice the last argument of the function is an instance of an (empty) type `DikjstraAlgorithm`. 

In [6]:
(d,p) = computeshortestpaths(dag, dag.nodes[1], DikjstraAlgorithm());

In [8]:
p

Dict{Int64, Union{Nothing, Int64}} with 5 entries:
  5 => 3
  4 => 5
  6 => 4
  2 => 1
  3 => 2

In [18]:
tmp_edge_list = Array{Tuple{Int64, Int64},1}();
for (s,t) ∈ p
    push!(tmp_edge_list, (t, s));
end
tmp_edge_list

5-element Vector{Tuple{Int64, Int64}}:
 (3, 5)
 (5, 4)
 (4, 6)
 (1, 2)
 (2, 3)

In [25]:
g = _convert(dag); # not a good idea ...
edgeweights = [g.weights[e.src, e.dst] for (_, e) in myedges]
@drawsvg begin
    background("black")
    sethue("white")
    drawgraph(g, layout=stress, vertexlabels = 1:nv(g),
    edgestrokecolors = (e, f, t, s, d) -> ((f,t) ∈ tmp_edge_list ? colorant"orange" : colorant"white"),
    edgestrokeweights = (n, from, to, edgesrc, edgedest) -> ((from, to) ∈ tmp_edge_list ? 2.0 : 0.5),
    edgelabels = edgeweights)
end 600 300

LoadError: type SimpleDiGraph has no field weights